# Paths

In [ ]:
local_analysis = Path("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/")

In [ ]:
csvs = local_analysis.glob("20230919_self_docked*/run_docking_oe.*-results.csv")

In [ ]:
df = pd.concat(map(pd.read_csv, csvs), ignore_index= True)

In [ ]:
df.groupby(["POSIT_method"]).mean()

In [ ]:
df.columns

# Cleanup

In [ ]:
df.columns = ["Compound_ID", 
                 "Structure_Source", 
                 "Docked_File", 
                 "Pose_ID", 
                 "RMSD", 
                 "POSIT", 
                 "POSIT_Method", 
                 "Chemgauss4", 
                 "Clash", 
                 "SMILES", 
                 "GAT_Score",
                "SCHNET_score"]

In [ ]:
df["POSIT_R"] = 1-df["POSIT"]
df["Complex_ID"] = df.Compound_ID.apply(str) + "_" + df.Structure_Source

## remove non-p series structures

In [ ]:
p_only = df[df.Structure_Source.str.contains("Mpro-P")]

In [ ]:
p_only.groupby(["POSIT_Method"]).count()

## get intersection

In [ ]:
p_only.groupby(["Compound_ID", "POSIT_Method"]).count()

In [ ]:
p_only[p_only.Compound_ID == "ALP-POS-133e7cd9-2"]

In [ ]:
print([file for file in p_only[p_only.Compound_ID == "ALP-POS-133e7cd9-2"]["Docked_File"]])

### ok somehow POSIT is reverting to using FRED when SHAPEFIT doesn't work

# get intersection

In [ ]:
cmpds = p_only[p_only.POSIT_Method == "SHAPEFIT"]["Compound_ID"]

In [ ]:
intersection = p_only[p_only.Compound_ID.isin(cmpds)]

In [ ]:
intersection.groupby(["POSIT_Method"]).count()

In [ ]:
cmpds = intersection[intersection.POSIT_Method == "FRED"]["Compound_ID"]

In [ ]:
intersection = intersection[intersection.Compound_ID.isin(cmpds)]

In [ ]:
intersection.groupby(["POSIT_Method"]).count()

In [ ]:
cmpds = intersection[intersection.POSIT_Method == "HYBRID"]["Compound_ID"]

In [ ]:
intersection = intersection[intersection.Compound_ID.isin(cmpds)]

In [ ]:
intersection.groupby(["POSIT_Method"]).count()

# save csv

In [ ]:
intersection.to_csv(local_analysis / "20230919_combined.csv")